In [1]:
from dataset.main import DatasetGenerator
import numpy as np
import sys

/Users/berkaychakar/opt/anaconda3/envs/FACT/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
dataset = DatasetGenerator(small=True, samples=10000, facts_per_person=10, distribution="zipf", place=False, day=False, dataset_folder='dataset/data/')
dataset.generate()

In [3]:
dataset.tokenize()

In [4]:
dataset.dataset[:20]

['Stephan had caesar salad',
 'Stephan had cheesecake',
 'Stephan had apple pie',
 'Stephan had bibimbap',
 'Stephan had baklava',
 'Stephan had fried calamari',
 'Stephan had lobster bisque',
 'Stephan had bread pudding',
 'Stephan had beet salad',
 'Stephan had club sandwich',
 'Darlene had apple pie',
 'Darlene had breakfast burrito',
 'Darlene had bibimbap',
 'Darlene had caprese salad',
 'Darlene had ravioli',
 'Darlene had grilled salmon',
 'Darlene had bruschetta',
 'Darlene had cheesecake',
 'Darlene had baklava',
 'Darlene had chocolate cake']

In [5]:
dataset.dataset_splitted[:20]

[['Stephan', 'caesar salad'],
 ['Stephan', 'cheesecake'],
 ['Stephan', 'apple pie'],
 ['Stephan', 'bibimbap'],
 ['Stephan', 'baklava'],
 ['Stephan', 'fried calamari'],
 ['Stephan', 'lobster bisque'],
 ['Stephan', 'bread pudding'],
 ['Stephan', 'beet salad'],
 ['Stephan', 'club sandwich'],
 ['Darlene', 'apple pie'],
 ['Darlene', 'breakfast burrito'],
 ['Darlene', 'bibimbap'],
 ['Darlene', 'caprese salad'],
 ['Darlene', 'ravioli'],
 ['Darlene', 'grilled salmon'],
 ['Darlene', 'bruschetta'],
 ['Darlene', 'cheesecake'],
 ['Darlene', 'baklava'],
 ['Darlene', 'chocolate cake']]

In [6]:
dataset.dataset_tokenized[:20]


[[0, 1],
 [0, 2],
 [0, 3],
 [0, 4],
 [0, 5],
 [0, 6],
 [0, 7],
 [0, 8],
 [0, 9],
 [0, 10],
 [11, 3],
 [11, 12],
 [11, 4],
 [11, 13],
 [11, 14],
 [11, 15],
 [11, 16],
 [11, 2],
 [11, 5],
 [11, 17]]

In [7]:
dataset.vocabulary_size

1095

In [8]:
import torch

## split the tokenized dataset into training and testing
dataset.split()

## get the training and testing datasets
train_dataset = [torch.tensor(x, dtype=torch.long) for x in dataset.train]
test_dataset =  [torch.tensor(x, dtype=torch.long) for x in dataset.test]

In [9]:
from torch.utils.data import Dataset

## create a dataset class
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Assuming each item in data is a sequence and we use the same sequence shifted by one as the target
        x = torch.tensor(self.data[idx][:-1], dtype=torch.long)
        y = torch.tensor(self.data[idx][1:], dtype=torch.long)
        return x, y


## create the datasets
train_data = MyDataset(train_dataset)
test_data = MyDataset(test_dataset)

print(train_data)

In [10]:
train_data[11]

/var/folders/rl/z562j0n16b75w5wm19c5jq7c0000gn/T/ipykernel_69791/2128282817.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(self.data[idx][:-1], dtype=torch.long)
/var/folders/rl/z562j0n16b75w5wm19c5jq7c0000gn/T/ipykernel_69791/2128282817.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.data[idx][1:], dtype=torch.long)


(tensor([977]), tensor([69]))

In [11]:
## import mingpt
sys.path.append('minGPT/')
from mingpt.model import GPT
from mingpt.utils import set_seed
set_seed(42)

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-mini'
model_config.vocab_size = dataset.vocabulary_size
model_config.block_size = 1
model = GPT(model_config)

number of parameters: 2.88M


In [12]:
# create a Trainer object
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
train_config.max_iters = 20000
train_config.num_workers = 0
trainer = Trainer(train_config, model, train_data)

running on device cpu


In [13]:
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
trainer.set_callback('on_batch_end', batch_end_callback)

trainer.run()

/Users/berkaychakar/opt/anaconda3/envs/FACT/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/rl/z562j0n16b75w5wm19c5jq7c0000gn/T/ipykernel_69791/2128282817.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(self.data[idx][:-1], dtype=torch.long)
/var/folders/rl/z562j0n16b75w5wm19c5jq7c0000gn/T/ipykernel_69791/2128282817.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.data[idx][1:], dtype=torch.long)


iter_dt 0.00ms; iter 0: train loss 7.00937
iter_dt 31.11ms; iter 100: train loss 3.94443
iter_dt 29.00ms; iter 200: train loss 3.83806
iter_dt 45.88ms; iter 300: train loss 3.78773
iter_dt 29.69ms; iter 400: train loss 4.03972
iter_dt 29.98ms; iter 500: train loss 3.87621
iter_dt 28.84ms; iter 600: train loss 3.85537
iter_dt 29.73ms; iter 700: train loss 3.83329
iter_dt 29.73ms; iter 800: train loss 3.70658
iter_dt 30.40ms; iter 900: train loss 3.73395
iter_dt 28.36ms; iter 1000: train loss 3.75842
iter_dt 29.69ms; iter 1100: train loss 3.63571
iter_dt 35.20ms; iter 1200: train loss 3.77344
iter_dt 28.77ms; iter 1300: train loss 3.52507
iter_dt 29.32ms; iter 1400: train loss 3.70121
iter_dt 29.46ms; iter 1500: train loss 3.66999
iter_dt 28.93ms; iter 1600: train loss 3.64048
iter_dt 28.59ms; iter 1700: train loss 3.57938
iter_dt 29.48ms; iter 1800: train loss 3.31754
iter_dt 29.26ms; iter 1900: train loss 3.55524
iter_dt 29.02ms; iter 2000: train loss 3.31474
iter_dt 29.24ms; iter 2100

In [14]:
# now let's perform some evaluation
model.eval()

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(1095, 192)
    (wpe): Embedding(1, 192)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x Block(
        (ln_1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=192, out_features=576, bias=True)
          (c_proj): Linear(in_features=192, out_features=192, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (mlp): ModuleDict(
          (c_fc): Linear(in_features=192, out_features=768, bias=True)
          (c_proj): Linear(in_features=768, out_features=192, bias=True)
          (act): NewGELU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_featur

In [15]:
## get the size of the testing dataset
n = len(test_data)

y_gen_all = []

## generate n sequences
for i in range(n):
    x, y = test_data[i]
    x = x.unsqueeze(0)
    decoded = dataset.decode(test_dataset[i].numpy())
    y_gen = model.generate(x, 1, do_sample=True)
    y_gen = y_gen.squeeze(0).cpu().numpy()
    y_gen = dataset.decode(y_gen)
    y_gen_all.append(y_gen)


## group the generated sequences by the first word
grouped = {}
for y_gen in y_gen_all:
    first_word = y_gen[0]
    if first_word not in grouped:
        grouped[first_word] = []
    grouped[first_word].append(y_gen)

## print the generated sequences
## check if it exists (accuracy)

wrong = 0
all = 0
for key in grouped:
    print(key)
    for seq in grouped[key]:
        print(seq)
        if seq not in dataset.dataset_splitted:
            wrong += 1

        all += 1
    print('\n')

print('wrong / all', wrong / all)

/var/folders/rl/z562j0n16b75w5wm19c5jq7c0000gn/T/ipykernel_69791/2128282817.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(self.data[idx][:-1], dtype=torch.long)
/var/folders/rl/z562j0n16b75w5wm19c5jq7c0000gn/T/ipykernel_69791/2128282817.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.data[idx][1:], dtype=torch.long)


Braden
['Braden', 'chicken curry']
['Braden', 'beef tartare']


Ronnica
['Ronnica', 'breakfast burrito']


Taddeo
['Taddeo', 'poutine']
['Taddeo', 'cheesecake']


Rickey
['Rickey', 'clam chowder']
['Rickey', 'clam chowder']
['Rickey', 'clam chowder']


Stanleigh
['Stanleigh', 'croque madame']
['Stanleigh', 'bread pudding']


Stephan
['Stephan', 'beet salad']
['Stephan', 'cheesecake']
['Stephan', 'cup cakes']
['Stephan', 'clam chowder']


Sam
['Sam', 'cheesecake']
['Sam', 'cheesecake']
['Sam', 'beignets']
['Sam', 'apple pie']


Wallas
['Wallas', 'baby back ribs']
['Wallas', 'beef carpaccio']
['Wallas', 'baklava']
['Wallas', 'beignets']


Ring
['Ring', 'lobster bisque']
['Ring', 'bibimbap']
['Ring', 'baby back ribs']
['Ring', 'edamame']


Marielle
['Marielle', 'apple pie']
['Marielle', 'cannoli']


Fee
['Fee', 'red velvet cake']


Mufi
['Mufi', 'cheese plate']
['Mufi', 'pizza']


Jard
['Jard', 'escargots']
['Jard', 'ceviche']


Olvan
['Olvan', 'baklava']
['Olvan', 'baklava']
['Olvan', 'r